In [ ]:
import anndata

from rp2 import create_folder, get_data_path, get_output_path, get_scripts_path, working_directory

As a proof of concept, create UMI .csv files for 10 genes per condition and run them through txburst scripts

In [ ]:
nouse_umi_ad = anndata.read_h5ad(get_data_path("ArrayExpress", "E-MTAB-6754.processed.2.mouse.h5ad"))

In [ ]:
txburst_files_path = get_output_path("txburst")
create_folder(txburst_files_path, create_clean=True)

for (replicate, treatment, time_point), df in nouse_umi_ad.obs.groupby(["replicate", "treatment", "time_point"]):
    csv_path = txburst_files_path.joinpath(f"mouse_umi_{replicate}_{treatment}_{time_point}.csv")
    subset_ad = nouse_umi_ad[df.index.values, :10]
    subset_ad.to_df().T.to_csv(csv_path, header=False)

In [ ]:
txburst_script_path = get_scripts_path("txburst")

with working_directory(txburst_files_path):
    for csv_file_path in txburst_files_path.glob("*.csv"):
        txburst_ml_script_path = txburst_script_path.joinpath("txburstML.py")
        cmd = f"{txburst_ml_script_path} --njobs 4 {csv_file_path.name}"
        print("Excecuting:", cmd)
        %run {cmd}